<a href="https://colab.research.google.com/github/madhu143binnu/Capstone_bank_analysis_new/blob/main/Banking_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [251]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()

--2023-12-19 13:05:39--  https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220272364 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop2.7.tgz.1’

spark-3.0.0-bin-had 100%[===================>] 210.07M  17.5MB/s    in 11s     

2023-12-19 13:05:50 (19.6 MB/s) - ‘spark-3.0.0-bin-hadoop2.7.tgz.1’ saved [220272364/220272364]

spark-3.0.0-bin-hadoop2.7/
spark-3.0.0-bin-hadoop2.7/NOTICE
spark-3.0.0-bin-hadoop2.7/kubernetes/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.0-bin-hadoop2

In [252]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DemoApp").master("local").getOrCreate()


In [3]:
import sqlite3
import pandas as pd

db = sqlite3.connect('customers.db')

query_create_table = 'CREATE TABLE customers(customer_id INTEGER, first_name VARCHAR, last_name VARCHAR, date_of_birth DATE)'
db.execute(query_create_table)

query_insert_data = '''
INSERT INTO customers (customer_id, first_name, last_name, date_of_birth) VALUES
(1, 'John', 'Doe', '1980-05-15'),
(2, 'Jane', 'Smith', '1992-08-21'),
(3, 'Alice', 'Johnson', '1975-02-10'),
(4, 'Sarah', 'Jones', '1988-12-03'),
(5, 'David', 'Brown', '1995-04-18'),
(6, 'Emma', 'Miller', '1982-07-25');
'''

db.execute(query_insert_data)

query_select_all = 'SELECT * FROM customers'
customers_df = pd.read_sql_query(query_select_all, db)
print(customers_df)

db.commit()


   customer_id first_name last_name date_of_birth
0            1       John       Doe    1980-05-15
1            2       Jane     Smith    1992-08-21
2            3      Alice   Johnson    1975-02-10
3            4      Sarah     Jones    1988-12-03
4            5      David     Brown    1995-04-18
5            6       Emma    Miller    1982-07-25


In [253]:
import json

json_file_path = "accounts.json"

with open(json_file_path, 'r') as file:
    json_data = json.load(file)

accounts = json_data["accounts"]

for accounts in accounts:
    print("account_id:", accounts["account_id"])
    print("customer_id:", accounts["customer_id"])
    print("employee_id:", accounts["employee_id"])
    print("account_type:", accounts["account_type"])
    print("balance:", accounts["balance"])
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder.appName("example").getOrCreate()
json_file_path = "accounts.json"
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
schema = StructType([
    StructField("account_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("employee_id", IntegerType(), True),
    StructField("account_type", StringType(), True),
])
rows = [(accounts["account_id"], accounts["customer_id"], accounts["employee_id"],accounts["account_type"]) for accounts in json_data["accounts"]]
accounts_df = spark.createDataFrame(rows, schema=schema)
accounts_df.show(truncate=False)


account_id: 1
customer_id: 1
employee_id: 2
account_type: Savings
balance: 5000
account_id: 2
customer_id: 1
employee_id: 3
account_type: Checking
balance: 1000
account_id: 3
customer_id: 2
employee_id: 4
account_type: Savings
balance: 8000
account_id: 4
customer_id: 3
employee_id: 1
account_type: Checking
balance: 3000
account_id: 5
customer_id: 2
employee_id: 2
account_type: Checking
balance: 2500
account_id: 6
customer_id: 3
employee_id: 3
account_type: Savings
balance: 6000
account_id: 7
customer_id: 4
employee_id: 4
account_type: Checking
balance: 12000
account_id: 8
customer_id: 5
employee_id: 2
account_type: Savings
balance: 3000
+----------+-----------+-----------+------------+
|account_id|customer_id|employee_id|account_type|
+----------+-----------+-----------+------------+
|1         |1          |2          |Savings     |
|2         |1          |3          |Checking    |
|3         |2          |4          |Savings     |
|4         |3          |1          |Checking    |
|5   

In [254]:
transactions_df = spark.read.csv(path='transactions.csv',
                             sep=",",
                             header=True)

transactions_df.show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|             3|         2|         Deposit|  2000|2023-03-10 15:20:00|
|             4|         3|      Withdrawal|  1000|2023-04-05 10:10:00|
|             5|         4|         Deposit|  1500|2023-05-20 09:00:00|
|             6|         5|         Deposit|  2000|2023-06-12 11:30:00|
|             7|         3|      Withdrawal|   800|2023-07-08 14:15:00|
|             8|         2|         Deposit|  3000|2023-08-22 16:45:00|
|             9|         4|      Withdrawal|  1500|2023-09-14 09:30:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------

In [255]:
import xml.etree.ElementTree as ET
import pandas as pd

tree = ET.parse('employees.xml')
root = tree.getroot()
data = []
for child in root:
    record = {}
    for subchild in child:
        record[subchild.tag] = subchild.text
    data.append(record)

employes_df = pd.DataFrame(data)
print(employes_df)


   employee_id branch_id first_name last_name position
0            1         2       Mike   Johnson  Manager
1            2         3      Emily  Williams   Teller
2            3         2     Robert     Davis   Teller
3            4         3     Olivia    Wilson   Teller
4            5         2     Daniel   Johnson  Analyst
5            6         3     Sophia     Clark  Manager
6            7         2       Mike   Johnson  Manager
7            8         3      Emily  Williams   Teller
8            9         2     Robert     Davis   Teller
9           10         3     Olivia    Wilson   Teller
10          11         2     Daniel   Johnson  Analyst
11          12         3     Sophia     Clark  Manager


In [256]:
loans_df = spark.read.csv(path='loans.csv',
                             sep=",",
                             header=True)
loans_df.show()


+-------+-----------+-----------+-------------+----------+----------+------+
|loan_id|customer_id|loan_amount|interest_rate|start_date|  end_date|status|
+-------+-----------+-----------+-------------+----------+----------+------+
|      1|          1|      10000|         0.05|2023-01-01|2023-12-31|Active|
|      2|          1|      15000|         0.04|2023-02-15|2023-12-31|Active|
|      3|          2|       8000|         0.03|2023-03-20|2023-11-30|Active|
|      4|          3|      20000|         0.05|2023-01-10|2023-10-31|Active|
+-------+-----------+-----------+-------------+----------+----------+------+



In [257]:
customers_df = spark.read.csv(path='customers.csv',
                             sep=",",
                             header=True)
customers_df.show()


+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|date_of_birth|
+-----------+----------+---------+-------------+
|          1|      John|      Doe|   1980-05-15|
|          2|      Jane|    Smith|   21-08-1992|
|          3|     Alice|  Johnson|   10-02-1975|
|          4|     Sarah|    Jones|   03-12-1988|
|          5|     David|    Brown|   18-04-1995|
|          6|      Emma|   Miller|   25-07-1982|
+-----------+----------+---------+-------------+



In [258]:
accounts_df = spark.read.csv(path='accounts.csv',
                             sep=",",
                             header=True)
accounts_df.show()


+----------+-----------+-----------+------------+-------+
|account_id|customer_id|employee_id|account_type|balance|
+----------+-----------+-----------+------------+-------+
|         1|          1|          2|     Savings|   5000|
|         2|          1|          3|    Checking|   1000|
|         3|          2|          4|     Savings|   8000|
|         4|          3|          1|    Checking|   3000|
|         5|          2|          2|    Checking|   2500|
|         6|          3|          3|     Savings|   6000|
|         7|          4|          4|    Checking|  12000|
|         8|          5|          2|     Savings|   3000|
+----------+-----------+-----------+------------+-------+



In [259]:
employees_df = spark.read.csv(path='employees.csv',
                             sep=",",
                             header=True)
employees_df.show()


+-----------+---------+----------+---------+--------+
|employee_id|branch_id|first_name|last_name|position|
+-----------+---------+----------+---------+--------+
|          1|        2|      Mike|  Johnson| Manager|
|          2|        3|     Emily| Williams|  Teller|
|          3|        2|    Robert|    Davis|  Teller|
|          4|        3|    Olivia|   Wilson|  Teller|
|          5|        2|    Daniel|  Johnson| Analyst|
|          6|        3|    Sophia|    Clark| Manager|
|          7|        2|      Mike|  Johnson| Manager|
|          8|        3|     Emily| Williams|  Teller|
|          9|        2|    Robert|    Davis|  Teller|
|         10|        3|    Olivia|   Wilson|  Teller|
|         11|        2|    Daniel|  Johnson| Analyst|
|         12|        3|    Sophia|    Clark| Manager|
+-----------+---------+----------+---------+--------+



In [260]:
branches_df = spark.read.csv(path='branches.csv',
                             sep=",",
                             header=True)
branches_df.show()


+---------+---------------+-------------+
|branch_id|    branch_name|     location|
+---------+---------------+-------------+
|        1|    Main Branch|Downtown City|
|        2|Suburban Branch|  Suburbville|
|        3|Regional Branch|Regional City|
+---------+---------------+-------------+



In [261]:
payment_history_df = spark.read.csv(path='payment_history.csv',
                             sep=",",
                             header=True)
payment_history_df.show()


+----------+-------+------------+-----------+
|payment_id|loan_id|payment_date|amount_paid|
+----------+-------+------------+-----------+
|      null|      1|  2023-03-01|       2000|
|      null|      1|  2023-04-01|       1500|
|      null|      2|  2023-04-10|       1000|
|      null|      3|  2023-02-20|       3000|
|      null|      3|  2023-03-15|       2500|
+----------+-------+------------+-----------+



In [262]:
import json

json_file_path = "branches.json"

with open(json_file_path, 'r') as file:
    json_data = json.load(file)

branches = json_data["branches"]

for branch in branches:
    print("Branch ID:", branch["branch_id"])
    print("Branch Name:", branch["branch_name"])
    print("Location:", branch["location"])
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import json

spark = SparkSession.builder.appName("example").getOrCreate()
json_file_path = "branches.json"
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
schema = StructType([
    StructField("branch_id", IntegerType(), True),
    StructField("branch_name", StringType(), True),
    StructField("location", StringType(), True),
])
rows = [(branch["branch_id"], branch["branch_name"], branch["location"]) for branch in json_data["branches"]]
branches_df = spark.createDataFrame(rows, schema=schema)
branches_df.show(truncate=False)


Branch ID: 1
Branch Name: Main Branch
Location: Downtown City
Branch ID: 2
Branch Name: Suburban Branch
Location: Suburbville
Branch ID: 3
Branch Name: Regional Branch
Location: Regional City
+---------+---------------+-------------+
|branch_id|branch_name    |location     |
+---------+---------------+-------------+
|1        |Main Branch    |Downtown City|
|2        |Suburban Branch|Suburbville  |
|3        |Regional Branch|Regional City|
+---------+---------------+-------------+



Write a spark dataframe to show the balance amount for an account_id = 1:


In [327]:
from pyspark.sql import functions as F

accounts_df.filter(accounts_df['account_id'] == 1).select('balance').show()



+-------+
|balance|
+-------+
|   5000|
+-------+



2.List Transactions for an account_id = 1:

In [29]:
transactions_df.filter(transactions_df['account_id'] == 1).show()


+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
+--------------+----------+----------------+------+-------------------+



3.List Accounts with a zero balance

In [325]:
accounts_df.filter(accounts_df['balance'] == 0).select('account_id').show()
accounts_df.filter(accounts_df['balance'] == 0).show()

+----------+
|account_id|
+----------+
+----------+

+----------+-----------+-----------+------------+-------+
|account_id|customer_id|employee_id|account_type|balance|
+----------+-----------+-----------+------------+-------+
+----------+-----------+-----------+------------+-------+



4.Find the Oldest Customer

In [23]:
customers_df.sort_values(by=['date_of_birth'], ascending=True).head(1)

,customer_id,first_name,last_name,date_of_birth
2,3,Alice,Johnson,1975-02-10


5. Calculate the Total Interest Earned Across All Accounts

In [267]:
# prompt: Calculate the Total Interest Earned Across All Accounts

interest_earned = transactions_df.agg(F.sum('amount').alias('total_interest'))
interest_earned.show()


+--------------+
|total_interest|
+--------------+
|       14500.0|
+--------------+



6.List All Accounts with Customer Information

In [30]:
df = accounts_df.join(customers_df,accounts_df.customer_id ==  customers_df.customer_id,"inner")
df.show()


+----------+-----------+-----------+------------+-------+-----------+----------+---------+-------------+
|account_id|customer_id|employee_id|account_type|balance|customer_id|first_name|last_name|date_of_birth|
+----------+-----------+-----------+------------+-------+-----------+----------+---------+-------------+
|         1|          1|          2|     Savings|   5000|          1|      John|      Doe|   1980-05-15|
|         2|          1|          3|    Checking|   1000|          1|      John|      Doe|   1980-05-15|
|         3|          2|          4|     Savings|   8000|          2|      Jane|    Smith|   21-08-1992|
|         4|          3|          1|    Checking|   3000|          3|     Alice|  Johnson|   10-02-1975|
|         5|          2|          2|    Checking|   2500|          2|      Jane|    Smith|   21-08-1992|
|         6|          3|          3|     Savings|   6000|          3|     Alice|  Johnson|   10-02-1975|
|         7|          4|          4|    Checking|  1200

7.Calculate Total Balance for Each Customer

In [297]:
from pyspark.sql.types import *

import pyspark.sql.functions as F
test = accounts_df.groupBy("customer_id").agg(F.sum('balance').alias('total_bal'))
test1 = test.join(customers_df, "customer_id", 'inner')
test_new= test1.orderBy(F.desc("total_bal")).show(5)


+-----------+---------+----------+---------+-------------+
|customer_id|total_bal|first_name|last_name|date_of_birth|
+-----------+---------+----------+---------+-------------+
|          4|  12000.0|     Sarah|    Jones|   03-12-1988|
|          2|  10500.0|      Jane|    Smith|   21-08-1992|
|          3|   9000.0|     Alice|  Johnson|   10-02-1975|
|          1|   6000.0|      John|      Doe|   1980-05-15|
|          5|   3000.0|     David|    Brown|   18-04-1995|
+-----------+---------+----------+---------+-------------+



8. Find Customers with Multiple Accounts

In [298]:
df = accounts_df.join(customers_df,accounts_df.customer_id ==  customers_df.customer_id,"inner").show()

accounts_df.groupby("customer_id").agg(F.count("account_id").alias("count")).filter(F.col('count') > 0).show()

multi_df = accounts_df.groupBy('customer_id').agg(F.count('account_id').alias('total_accounts'))
tb_df = multi_df.filter(F.col('total_accounts')>1)
tb_df.show()

+----------+-----------+-----------+------------+-------+-----------+----------+---------+-------------+
|account_id|customer_id|employee_id|account_type|balance|customer_id|first_name|last_name|date_of_birth|
+----------+-----------+-----------+------------+-------+-----------+----------+---------+-------------+
|         1|          1|          2|     Savings|   5000|          1|      John|      Doe|   1980-05-15|
|         2|          1|          3|    Checking|   1000|          1|      John|      Doe|   1980-05-15|
|         3|          2|          4|     Savings|   8000|          2|      Jane|    Smith|   21-08-1992|
|         4|          3|          1|    Checking|   3000|          3|     Alice|  Johnson|   10-02-1975|
|         5|          2|          2|    Checking|   2500|          2|      Jane|    Smith|   21-08-1992|
|         6|          3|          3|     Savings|   6000|          3|     Alice|  Johnson|   10-02-1975|
|         7|          4|          4|    Checking|  1200

2. Customer Transactions Reports
1.List Transactions with Account and Customer Information


In [35]:
transactions_df.join(accounts_df, transactions_df.account_id == accounts_df.account_id, 'inner').join(customers_df, accounts_df.customer_id == customers_df.customer_id, 'inner').show()


+--------------+----------+----------------+------+-------------------+----------+-----------+-----------+------------+-------+-----------+----------+---------+-------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|account_id|customer_id|employee_id|account_type|balance|customer_id|first_name|last_name|date_of_birth|
+--------------+----------+----------------+------+-------------------+----------+-----------+-----------+------------+-------+-----------+----------+---------+-------------+
|             1|         1|         Deposit|  1000|2023-01-15 08:30:00|         1|          1|          2|     Savings|   5000|          1|      John|      Doe|   1980-05-15|
|             2|         1|      Withdrawal|   500|2023-02-02 12:45:00|         1|          1|          2|     Savings|   5000|          1|      John|      Doe|   1980-05-15|
|             3|         2|         Deposit|  2000|2023-03-10 15:20:00|         2|          1|          3|    Checking|   100

2. Calculate Average Transaction Amount

In [280]:
transactions_df.agg(F.avg("amount").alias("avg_transaction_amount")).show(truncate=False)

+----------------------+
|avg_transaction_amount|
+----------------------+
|1450.0                |
+----------------------+



3. Identify High-Value Customers with Total Balance

In [305]:
import pyspark.sql.functions as F
df_new = accounts_df.join(customers_df, on = 'customer_id', how = 'inner')
result = df_new.select('customer_id', 'first_name', 'last_name', 'account_id', 'account_type', 'balance')
result.show()

test = accounts_df.groupBy("customer_id").agg(F.sum('balance').alias('total_bal'))
test1 = test.join(customers_df, "customer_id", 'inner')
test_new= test1.orderBy(F.desc("total_bal")).show(5)

min_df = accounts_df.groupBy("customer_id").agg(F.min('balance').alias('min_bal'))
test_new1= min_df.orderBy(F.desc("min_bal")).show(5)

join_df1 = result.groupBy("")


+-----------+----------+---------+----------+------------+-------+
|customer_id|first_name|last_name|account_id|account_type|balance|
+-----------+----------+---------+----------+------------+-------+
|          1|      John|      Doe|         1|     Savings|   5000|
|          1|      John|      Doe|         2|    Checking|   1000|
|          2|      Jane|    Smith|         3|     Savings|   8000|
|          3|     Alice|  Johnson|         4|    Checking|   3000|
|          2|      Jane|    Smith|         5|    Checking|   2500|
|          3|     Alice|  Johnson|         6|     Savings|   6000|
|          4|     Sarah|    Jones|         7|    Checking|  12000|
|          5|     David|    Brown|         8|     Savings|   3000|
+-----------+----------+---------+----------+------------+-------+

+-----------+---------+----------+---------+-------------+
|customer_id|total_bal|first_name|last_name|date_of_birth|
+-----------+---------+----------+---------+-------------+
|          4|  120

In [48]:

accounts_df.groupby("customer_id").agg(F.sum("balance").alias("total_balance")).orderBy("total_balance",ascending=False).show()


+-----------+-------------+
|customer_id|total_balance|
+-----------+-------------+
|          4|      12000.0|
|          2|      10500.0|
|          3|       9000.0|
|          1|       6000.0|
|          5|       3000.0|
+-----------+-------------+



4.List Employees and Their Assigned Customers

In [312]:
df_new = accounts_df.join(customers_df, on = 'customer_id', how = 'inner')
#df_emp2 = spark.createDataFrame(employees_df)

lec_df = df_new.join(employees_df, on = 'employee_id', how = 'inner')

lec_df.show()

+-----------+-----------+----------+------------+-------+----------+---------+-------------+---------+----------+---------+--------+
|employee_id|customer_id|account_id|account_type|balance|first_name|last_name|date_of_birth|branch_id|first_name|last_name|position|
+-----------+-----------+----------+------------+-------+----------+---------+-------------+---------+----------+---------+--------+
|          2|          1|         1|     Savings|   5000|      John|      Doe|   1980-05-15|        3|     Emily| Williams|  Teller|
|          3|          1|         2|    Checking|   1000|      John|      Doe|   1980-05-15|        2|    Robert|    Davis|  Teller|
|          4|          2|         3|     Savings|   8000|      Jane|    Smith|   21-08-1992|        3|    Olivia|   Wilson|  Teller|
|          1|          3|         4|    Checking|   3000|     Alice|  Johnson|   10-02-1975|        2|      Mike|  Johnson| Manager|
|          2|          2|         5|    Checking|   2500|      Jane| 

5. Calculate the Total Number of Transactions for Each Account Type

In [314]:

total_transactions_by_type = joined_df.groupBy("account_type").agg(count("transaction_id").alias("total_transactions"))

total_transactions_by_type.show()

+------------+------------------+
|account_type|total_transactions|
+------------+------------------+
|     Savings|                 5|
|    Checking|                 5|
+------------+------------------+



6. Find Customers with No Accounts

In [315]:
customers_alias = customers_df.alias("c")
accounts_alias = accounts_df.alias("a")

customers_with_no_accounts = customers_alias.join(accounts_alias, col("c.customer_id") == col("a.customer_id"), "left_anti")

customers_with_no_accounts.show()

+-----------+----------+---------+-------------+
|customer_id|first_name|last_name|date_of_birth|
+-----------+----------+---------+-------------+
|          6|      Emma|   Miller|   25-07-1982|
+-----------+----------+---------+-------------+



In [94]:
transactions_df.groupby("transaction_date").agg(F.count("account_id").alias("account_id")).show()


+-------------------+----------+
|   transaction_date|account_id|
+-------------------+----------+
|2023-02-02 12:45:00|         1|
|2023-01-15 08:30:00|         1|
|2023-10-01 10:00:00|         1|
|2023-07-08 14:15:00|         1|
|2023-05-20 09:00:00|         1|
|2023-04-05 10:10:00|         1|
|2023-03-10 15:20:00|         1|
|2023-09-14 09:30:00|         1|
|2023-08-22 16:45:00|         1|
|2023-06-12 11:30:00|         1|
+-------------------+----------+



7.Find Customers with No Accounts

In [101]:
customers_df.join(accounts_df, "customer_id", "outer").filter(F.col("account_id").isNull()).show()


+-----------+----------+---------+-------------+----------+-----------+------------+-------+
|customer_id|first_name|last_name|date_of_birth|account_id|employee_id|account_type|balance|
+-----------+----------+---------+-------------+----------+-----------+------------+-------+
|          6|      Emma|   Miller|   25-07-1982|      null|       null|        null|   null|
+-----------+----------+---------+-------------+----------+-----------+------------+-------+



8. List the Latest Transaction for Each Account

In [324]:
#transactions_df.groupby("account_id",).agg(F.max("transaction_date").alias("latest_transaction")).show()
from pyspark.sql.types import *
from pyspark.sql.functions import*
from pyspark.sql import *
window_spec = Window.partitionBy("account_id").orderBy(F.desc("transaction_date"))

df_transactions_with_row_number = transactions_df.withColumn("row_number", F.row_number().over(window_spec))

latest_transactions = df_transactions_with_row_number.filter("row_number = 1")

latest_transactions = latest_transactions.drop("row_number")

latest_transactions.show()

+--------------+----------+----------------+------+-------------------+
|transaction_id|account_id|transaction_type|amount|   transaction_date|
+--------------+----------+----------------+------+-------------------+
|             7|         3|      Withdrawal|   800|2023-07-08 14:15:00|
|             6|         5|         Deposit|  2000|2023-06-12 11:30:00|
|            10|         1|         Deposit|  1200|2023-10-01 10:00:00|
|             9|         4|      Withdrawal|  1500|2023-09-14 09:30:00|
|             8|         2|         Deposit|  3000|2023-08-22 16:45:00|
+--------------+----------+----------------+------+-------------------+



9. Calculate the Total Withdrawals for Each Customer

In [328]:
trans_df = df_new.join(transactions_df, 'account_id')
trans_df2 = trans_df[trans_df['transaction_type'] == 'Withdrawal']
trans_df3 = trans_df2.select('customer_id', 'first_name', 'last_name', 'amount')
trans_df4 = trans_df3.groupBy('customer_id', 'first_name', 'last_name').agg(F.sum('amount').alias('total_withdrwals'))
trans_df4.show()

+-----------+----------+---------+----------------+
|customer_id|first_name|last_name|total_withdrwals|
+-----------+----------+---------+----------------+
|          2|      Jane|    Smith|          1800.0|
|          1|      John|      Doe|           500.0|
|          3|     Alice|  Johnson|          1500.0|
+-----------+----------+---------+----------------+



10. Find Duplicate Transactions

In [329]:

df = transactions_df.groupby('transaction_id').count().alias("duplicate_count")
df_new = df.filter(col("count" )> 1).alias("duplicate")
df_new.show()



+--------------+-----+
|transaction_id|count|
+--------------+-----+
+--------------+-----+

